# 8-puzzle
**<div style="text-align: right"> [Total score: 75]</div>**
N-Puzzle or sliding puzzle is a popular puzzle that consists of N tiles where N can be 8, 15, 24 and so on. In our example N = 8. The puzzle is divided into sqrt(N+1) rows and sqrt(N+1) columns. Eg. 15-Puzzle will have 4 rows and 4 columns and an 8-Puzzle will have 3 rows and 3 columns. The puzzle consists of N tiles and one empty space where the tiles can be moved. Start and Goal configurations (also called state) of the puzzle are provided. The puzzle can be solved by moving the tiles one by one in the single empty space and thus achieving the Goal configuration.

![8+puzzle](https://docs.google.com/drawings/d/e/2PACX-1vT15wXMNqd2zIfnf2CIS73dd3_zKu1wXVgHE8uaOLDNNF-j04reoX37GR8q71F9NwD94pzaZYjkbuSD/pub?w=344&h=158)


In this project, you will need to solve the 8-puzzle using BFS, DFS and A star algorithm.
As an example, in the figure above, the initial state is `[1,8,2,0,4,3,7,6,5]`. We need to obtain `[0,1,2,3,4,5,6,7,8]` in the final state.

Suppose if the initial state is `[3,1,2,0,4,5,6,7,8]`

The path followed by all BFS, DFS and A-star to reach the goal state is `['Up']`.

You will need to return the path in each of the exercises. The path returned should be in the format: `['Up', 'Down', 'Right', 'Left', 'Up', 'Left']` i.e. you are required to use first letter as Capital for each action and return the actions in a list. Also note that whenever an arbitary choice has to be made we choose the next node in UDLR (Up, Down, Left, Right) order.

**You also need to test whether a node is goal state or not only after the node has been popped from the frontier. Otherwise your answer may deviate. Also, only push those nodes to frontier which has not been yet added to the frontier (and hence explored)".**

A class has been constructed to represent the puzzle as below.


In [ ]:
import sys
import math
import resource

import numpy as np

## A Class to represent 8 puzzle.
class PuzzleBoard:

    def __init__(self, puzzle_state, parent=None, state="Initial"):
        self.parent = parent
        self.children = []
        self.puzzle_state = np.array(puzzle_state)
        self.column_size = int(math.sqrt(len(puzzle_state)))
        self.state = state
        self.depth = 0 if parent is None else parent.depth + 1
        self.cost = self.depth + self.get_manhattan_distance
    
    # Comparing two nodes according to estimated cost and UDLR order if two nodes have same cost.
    def __lt__(self, other):
        if self.cost != other.cost:
            return self.cost < other.cost
        else:
            op = {'Up': 0, 'Down': 1, 'Left': 2, 'Right': 3}
            return op[self.state] < op[other.state]

    @property
    def goal_test(self):
        if np.array_equal(self.puzzle_state, np.arange(9)):
            return True

    @property
    def get_manhattan_distance(self):
        for index, number in enumerate(self.puzzle_state):
            return abs(index // 3 - number // 3) + abs(index % 3 - number % 3)

    def move_up(self, i):
        if i - self.column_size >= 0:
            puzzle_new, parent = self.swap(i, i - 3)
            return PuzzleBoard(puzzle_new, parent, state='Up')

    def move_down(self, i):
        if i + self.column_size <= len(self.puzzle_state) - 1:
            puzzle_new, parent = self.swap(i, i + 3)
            return PuzzleBoard(puzzle_new, parent, state='Down')

    def move_left(self, i):
        if i % self.column_size > 0:
            puzzle_new, parent = self.swap(i, i - 1)
            return PuzzleBoard(puzzle_new, parent, state='Left')

    def move_right(self, i):
        if i % self.column_size < self.column_size - 1:
            puzzle_new, parent = self.swap(i, i + 1)
            return PuzzleBoard(puzzle_new, parent, state='Right')

    def swap(self, index_one, index_two):
        puzzle_new = self.puzzle_state.copy()
        puzzle_new[index_one], puzzle_new[index_two] = puzzle_new[index_two], puzzle_new[index_one]
        return puzzle_new, self

    @property
    def print_puzzle(self):
        m = 0
        while (m < 9):
            print()
            print(str(self.puzzle_state[m]) +
                  ' ' +
                  str(self.puzzle_state[m +
                                        1]) +
                  ' ' +
                  str(self.puzzle_state[m +
                                        2]))
            m += 3
        print()
    
    # Expanding a node.
    @property
    def expand(self):
        x = list(self.puzzle_state).index(0)
        self.children.append(self.move_up(x))
        self.children.append(self.move_down(x))
        self.children.append(self.move_left(x))
        self.children.append(self.move_right(x))
        self.children = list(filter(None, self.children))
        return self.children

## Exercise 1: Breadth First Search  
**<div style="text-align: right"> [Score: 25]</div>**

**Task:** Find the path followed by BFS to reach the goal state.

Note: Your function should take an instance of class PuzzleBoard as input and return path followed by bfs search and number of nodes expanded during the search. 

In [ ]:
from collections import deque

puzzle_state = [3, 1, 2, 0, 4, 5, 6, 7, 8]
puzzle_board = PuzzleBoard(puzzle_state)

def bfs(puzzle_board):
    soln = None
    path = None
    nodes_expanded = 0
    max_depth = 0

    def ancestral_chain():
        current = soln
        chain = [current]
        while current.parent != None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path():
        path = [node.state for node in ancestral_chain()[-2::-1]]
        return path
    
    
    frontier = deque()
    frontier.append(puzzle_board)
    froxplored = set()
    while frontier:
        board = frontier.popleft()
        froxplored.add(tuple(board.puzzle_state))
        if board.goal_test:
            soln = board
            path = path()
            nodes_expanded = len(froxplored)-len(frontier)-1
            return nodes_expanded, path
        
        for neighbor in board.expand:
            if tuple(neighbor.puzzle_state) not in froxplored:
                frontier.append(neighbor)
                froxplored.add(tuple(neighbor.puzzle_state))
                max_depth = max(max_depth, neighbor.depth)
    return nodes_expanded, path


        
print(bfs(puzzle_board))
print(bfs(PuzzleBoard([1,2,5,3,4,0,6,7,8])))

In [ ]:
assert bfs(PuzzleBoard([3,1,2,0,4,5,6,7,8])) == (1, ['Up'])
assert bfs(PuzzleBoard([1,2,5,3,4,0,6,7,8])) == (10, ['Up', 'Left', 'Left'])

## Exercise 2: Depth First Search
**<div style="text-align: right"> [Score: 25]</div>**
Find the path followed by DFS to reach the goal state.

Note: Your function should take an instance of class PuzzleBoard as input and return path followed by dfs search and number of nodes expanded during the search. 

In [ ]:
def dfs(puzzle_board):
    soln = None
    path = None
    nodes_expanded = 0
    max_depth = 0

    def ancestral_chain():
        current = soln
        chain = [current]
        while current.parent != None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path():
        path = [node.state for node in ancestral_chain()[-2::-1]]
        return path

    frontier = []
    frontier.append(puzzle_board)
    froxplored = set()
    while frontier:
        board = frontier.pop()
        froxplored.add(tuple(board.puzzle_state))
        if board.goal_test:
            soln = board
            path = path()
            nodes_expanded = len(froxplored)-len(frontier)-1
            return nodes_expanded, path
        
        for neighbor in board.expand[::-1]:
            if tuple(neighbor.puzzle_state) not in froxplored:
                frontier.append(neighbor)
                froxplored.add(tuple(neighbor.puzzle_state))
                max_depth = max(max_depth, neighbor.depth)
    return None

print(dfs(PuzzleBoard([3,1,2,0,4,5,6,7,8])))
print(dfs(PuzzleBoard([1,3,0,4,2,5,6,7,8])))

In [ ]:
assert dfs(PuzzleBoard([3,1,2,0,4,5,6,7,8])) == (1, ['Up'])
assert dfs(PuzzleBoard([1,3,0,4,2,5,6,7,8])) == (402, ['Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Left', 'Up', 'Right', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Right', 'Up', 'Up', 'Left', 'Down', 'Right', 'Up', 'Left', 'Down', 'Down', 'Right', 'Right', 'Up', 'Up', 'Left', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Up', 'Right', 'Down', 'Down', 'Left', 'Up', 'Right', 'Up', 'Left'])

## Exercise 3: A Star Search
A* search evaluates nodes by combining cost to reach the node and cost estimate to get from the node to goal node; i.e. $f(n) = g(n) + h(n) = $ estimated cost of cheapest solution through node $n$, where  $g(n)$ is the cost of getting to node n from start node and $h(n)$ is the estimated cost from node n to the goal. In following task, you are required to use manhattan distance as heuristic. Manhattan distance for each tile is the minimumm number of moves required to place that tile in required position. In the example board given above, manhattan distance for tile 5 is 2 and for tile 2 is 0. Total manhattan distance for the puzzle state is 11.

**<div style="text-align: right"> [Score: 25]</div>**

Find the path followed by A-star to reach the goal state.

Note: Your function should take an instance of class PuzzleBoard as input and return path followed by A* search and number of nodes expanded during the search. Number of nodes expanded might slightly differ for A* search according to implementation. 

Hint: You can use manhattan distance given in class description code given above.

In [ ]:
import heapq

def ast(puzzle_board):
    soln = None
    path = None
    nodes_expanded = 0
    max_depth = 0

    def ancestral_chain():
        current = soln
        chain = [current]
        while current.parent != None:
            chain.append(current.parent)
            current = current.parent
        return chain

    def path():
        path = [node.state for node in ancestral_chain()[-2::-1]]
        return path
    
    frontier = []
    heapq.heappush(frontier, puzzle_board)
    froxplored = set()
    while frontier:
        board = heapq.heappop(frontier)
        froxplored.add(tuple(board.puzzle_state))
        if board.goal_test:
            soln = board
            path = path()
            nodes_expanded = len(froxplored)-len(frontier)-1
            return nodes_expanded, path
        
        for neighbor in board.expand:
            if tuple(neighbor.puzzle_state) not in froxplored:
                heapq.heappush(frontier, neighbor)
                froxplored.add(tuple(neighbor.puzzle_state))
                self = max(max_depth, neighbor.depth)
    return None

ast(PuzzleBoard([1,2,5,3,4,0,6,7,8]))

In [ ]:
assert ast(PuzzleBoard([3,1,2,0,4,5,6,7,8]))[1] == ['Up']
assert ast(PuzzleBoard([1,2,5,3,4,0,6,7,8]))[1] == ['Up', 'Left', 'Left']